In [1]:
#ライブラリ読み込み
from transformers import BertJapaneseTokenizer, BertModel
import torch
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from janome.tokenizer import Tokenizer
import re
import seaborn as sns
%matplotlib inline
from transformers import *
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import torch.backends.cudnn as cudnn
import random
import scipy.spatial
from sentence_transformers import SentenceTransformer, util

C:\Users\kimpa\anaconda3\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


# データ読み込み

In [3]:
DATA_IN_PATH = "C:/r_analysis/1.20221101/"
train=pd.read_csv(DATA_IN_PATH + 'data_complete_34814.csv',header=0,encoding="cp932")
data=train['review_cleansing']
data=pd.Series(data)
data=data.tolist()

# Sentence-BERT

In [4]:

class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)



In [5]:

MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"  # <- v2です。
model = SentenceBertJapanese(MODEL_NAME)


loading file https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2/resolve/main/vocab.txt from cache at C:\Users\kimpa/.cache\huggingface\transformers\ebbda1022f98b7e946d3ef6d0f741839e781bc7e0473706b7a6507a84434caee.a7378a0cbee5cff668832a776d72b97a25479604fe9564d5595897f75049e7f4
loading file https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2/resolve/main/special_tokens_map.json from cache at C:\Users\kimpa/.cache\huggingface\transformers\a5a03392304e8c829ac0f9db58c32b9557ae97cc36a346a2d0bf2f39bf3ce642.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2/resolve/main/tokenizer_config.json from cache at C:\Users\kimpa/.cache\huggingface\transformers\f184397309e60b06383c3ecb1c0d72b957c101b45068ee5e2e853e36b6b02e06.23b7e22423c9bd0f17314f2

# 文章ベクトル読み込み

In [6]:
DATA_IN_PATH_SURVEY = "C:/r_analysis/1.20221101/5.numpy_survey/"
DATA_IN_PATH_SEICHO = "C:/r_analysis/1.20221101/4.numpy_review/"
DATA_IN_PATH = "C:/r_analysis/1.20221101/"

In [8]:
#レビューデータのベクトル読み込み
sentence_vectors_seicho= np.load(DATA_IN_PATH_SEICHO+'complete_34814.npy')

#設問項目のベクトル読み込み
files = os.listdir(DATA_IN_PATH + "/3.survey_text/")
files1 = os.listdir(DATA_IN_PATH + "/5.numpy_survey/")


for i in files:
     globals()[i.split('.')[0]+str("_query")] = pd.read_csv(DATA_IN_PATH + "/3.survey_text/"+i,header=0,encoding="cp932")
        
for i in files1:
    globals()[i.split('.')[0]] = np.load(DATA_IN_PATH_SURVEY+ i)

In [10]:
#確認
for i in files:
    print("==================================================================")
    print('項目名 : {}'.format(i.split('.')[0]))
    print('項目数 : {}'.format(len(globals()[i.split('.')[0]+str("_query")])))
    print('設問内容')
    print(globals()[i.split('.')[0]+str("_query")]['q'])
    print("-------------------------------------------------------------------")
    print('embeddingベクトル')
    print(globals()[i.split('.')[0]])
    print("==================================================================")

항목명1 : CM_001
항목수 : 7
설문리스트
0           仕事生活の残りを今の会社で過ごせたらとても幸せだ
1            会社の外の人に自分の会社のことを話すのが楽しい
2    この会社の問題をまるで自分自身の問題であるかのように感じている
3              会社という家族の一員になっているように思う
4                   この会社に愛情を感じていると思う
5          この会社は私にとって個人的に重要な意味を持っている
6             自分の会社の一員なのだと強く感じることがある
Name: q, dtype: object
-------------------------------------------------------------------
엠베딩벡터값
[[-0.25652558  0.7605516  -0.3163144  ...  1.2289509  -0.08685125
  -0.18210262]
 [-0.2167822  -0.03472124  0.2513897  ...  0.01958281 -0.96365887
   0.31388605]
 [ 0.42096493  0.2829342  -0.17844585 ...  0.935098    0.14363
   0.03774778]
 ...
 [-0.5979044   0.45443568 -0.27390715 ...  0.5848064  -0.10046559
  -0.93840545]
 [ 0.8903785   0.60978997 -0.66844815 ... -0.96348596 -0.95518684
   0.05216397]
 [ 0.07852855  0.20395797 -0.45646304 ...  0.67746276 -0.4970082
  -0.21662432]]
항목명1 : CM_002
항목수 : 7
설문리스트
0                                 この会社を辞めるなんて自分に考えられない
1                  たとえこの会社を今すぐに辞めようと思っても実際に辞

KeyError: 'ENGAGEMENT'

# 類似度分析

## コサイン類似度が高い順に並べる

In [27]:
for query, query_embedding in zip(CM_001_query['q'], CM_001):
    d = scipy.spatial.distance.cdist([query_embedding], sentence_vectors_seicho, metric="cosine")[0] #코사인거리 계산
    e=1-d

    results = zip(range(len(e)), e) #コサイン類似度集計
    results = sorted(results, key=lambda x: x[1],reverse=True) #リスト化

    print("\n================================================================================================================================\n")
    print("項目:", query)
    print("                 ")
    for idx, distance in results[0:2]: #何番目まで表示するか指定
        print(data[idx].strip(), "(Score: %.4f)" % (distance))
print("\n================================================================================================================================\n")




項目: 仕事生活の残りを今の会社で過ごせたらとても幸せだ
                 
なし福利厚生は本当にいい定年まで現場で動けるのであれは最高だとは思います (Score: 0.6935)
日々の生活を守る会社の一員として社会貢献しているという気持ちで勤務できたことは良かったです (Score: 0.6785)


項目: 会社の外の人に自分の会社のことを話すのが楽しい
                 
バリバリの営業職お客さんと話すことや自分の力で成果を挙げることを楽しいと感じる人 (Score: 0.6860)
お客さんと話すのが好きな人売り場を自分で作って売上が跳ね上がった時などはかなり楽しいです (Score: 0.6842)


項目: この会社の問題をまるで自分自身の問題であるかのように感じている
                 
何かあると全ての責任を負わされます自分で考えて行動したから当たり前な会社ですどう取るかは本人の気持ちです (Score: 0.6731)
責任感はとても育ちます嫌でも育ちますお店の責任は全て自分にかかってきますので (Score: 0.6393)


項目: 会社という家族の一員になっているように思う
                 
資格取得は全部会社で負担この会社で働いて人と人とのつながりの大切さを学べた人を大切にすると自分に良いことがある今の職場でいかせてますこの会社にくらべ育児介護今は摘めたい会社です採用も子育て終えた50代を採用子供が小さい若い人は休みをとるかもしれないからダメと上はいうこの会社の子育てに対して若いママでも頑張れるていうのを子育てママに冷たい今の会社をかえていきたいこの会社のようにと今見習い今の会社をかえたい (Score: 0.6238)
名が知れた企業なので友人親族などにすごいと思ってもらえる点ぐらいです革新的なイメージがあると思いますがいたって普通の会社なのでいたって普通の会社が運営していくために働いている人が大半だということを忘れないでほしいです (Score: 0.6228)


項目: この会社に愛情を感じていると思う
                 
お客様に喜んでいただいけるとやりがいを感じることがある (Score: 0.7208)
社会や顧客の役に立っていると

## コサイン類似度だけ算出

In [14]:
DATA_OUT_PATH = 'C:/r_analysis/1.20221101/6.similarity/'

In [15]:
cosine_sim = util.pytorch_cos_sim(CM_001, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['CM01_01','CM01_02','CM01_03','CM01_04','CM01_05','CM01_06','CM01_07']
cosine_sim.to_csv(DATA_OUT_PATH + 'CM_01.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)

        CM01_01   CM01_02   CM01_03   CM01_04   CM01_05   CM01_06   CM01_07
0      0.358681  0.178321  0.435077  0.272600  0.311845  0.292773  0.331626
1      0.410611  0.343696  0.334813  0.396275  0.393568  0.410069  0.473619
2      0.438531  0.333019  0.361114  0.434440  0.448463  0.425638  0.519067
3      0.478306  0.361205  0.349161  0.359096  0.490814  0.461305  0.514312
4      0.403609  0.158250  0.334524  0.384020  0.419057  0.351953  0.446974
...         ...       ...       ...       ...       ...       ...       ...
34809  0.176042  0.374078  0.072624  0.162651  0.217228  0.136812  0.246095
34810  0.332392  0.242123  0.164486  0.312511  0.331925  0.312082  0.361494
34811  0.408173  0.278271  0.243791  0.353280  0.454682  0.395760  0.442346
34812  0.344964  0.146120  0.492269  0.289013  0.327928  0.293874  0.427159
34813  0.179293  0.369576  0.383125  0.178107  0.238091  0.226427  0.285572

[34814 rows x 7 columns]


In [17]:
cosine_sim = util.pytorch_cos_sim(CM_002, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['CM02_01','CM02_02','CM02_03','CM02_04','CM02_05','CM02_06','CM02_08']
cosine_sim.to_csv(DATA_OUT_PATH + 'CM_02.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)


        CM02_01   CM02_02   CM02_03   CM02_04   CM02_05   CM02_06   CM02_08
0      0.494074  0.505936  0.531629  0.549199  0.499005  0.571113  0.525312
1      0.329808  0.373059  0.310772  0.305115  0.546007  0.357879  0.437870
2      0.463570  0.505693  0.470270  0.445652  0.674438  0.483031  0.587742
3      0.387799  0.379436  0.342047  0.357218  0.555357  0.393153  0.496308
4      0.454201  0.468716  0.391007  0.442179  0.554544  0.506737  0.556854
...         ...       ...       ...       ...       ...       ...       ...
34809  0.150361  0.186388  0.135020  0.091284  0.250614  0.154031  0.063465
34810  0.186063  0.223494  0.146086  0.149195  0.350181  0.225812  0.246086
34811  0.226662  0.273323  0.205245  0.250706  0.389185  0.279821  0.272944
34812  0.566081  0.588922  0.490752  0.540531  0.581118  0.561620  0.476551
34813  0.306023  0.276865  0.168285  0.186626  0.324776  0.224658  0.239212

[34814 rows x 7 columns]


In [19]:
cosine_sim = util.pytorch_cos_sim(CM_003, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['CM03_01','CM03_02','CM03_03','CM03_04','CM03_05','CM03_07','CM03_08']
cosine_sim.to_csv(DATA_OUT_PATH + 'CM_03.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)

        CM03_01   CM03_02   CM03_03   CM03_04   CM03_05   CM03_07   CM03_08
0      0.457736  0.178814  0.406696  0.344372  0.391996  0.296688  0.325907
1      0.411290  0.414169  0.363333  0.507784  0.457641  0.436029  0.548654
2      0.589487  0.381662  0.472666  0.425277  0.534323  0.466035  0.525429
3      0.392082  0.316514  0.327293  0.448457  0.472920  0.452972  0.578923
4      0.449151  0.320370  0.371945  0.473006  0.517479  0.366622  0.449238
...         ...       ...       ...       ...       ...       ...       ...
34809  0.300614  0.154831  0.063929  0.278491  0.187270  0.219407  0.422417
34810  0.297801  0.337752  0.160788  0.456285  0.296238  0.344463  0.546643
34811  0.323664  0.278727  0.221002  0.393055  0.352873  0.486010  0.553865
34812  0.521427  0.310933  0.470174  0.435259  0.466511  0.378607  0.377375
34813  0.240856  0.260806  0.252206  0.222429  0.284838  0.220320  0.366596

[34814 rows x 7 columns]


In [21]:
cosine_sim = util.pytorch_cos_sim(DP_001, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['DP01_01','DP01_02','DP01_03','DP01_04']
cosine_sim.to_csv(DATA_OUT_PATH + 'DP01.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)

        DP01_01   DP01_02   DP01_03   DP01_04
0      0.089762  0.242829  0.101872  0.401408
1      0.375554  0.570494  0.403375  0.595043
2      0.263739  0.509605  0.199599  0.490457
3      0.202556  0.454835  0.238817  0.589340
4      0.168506  0.410989  0.102225  0.378331
...         ...       ...       ...       ...
34809  0.151238  0.345644  0.348631  0.355392
34810  0.200650  0.448448  0.253060  0.381679
34811  0.301051  0.358525  0.267602  0.346572
34812  0.198464  0.427667  0.204615  0.310747
34813  0.355698  0.432933  0.392806  0.195937

[34814 rows x 4 columns]


In [23]:
cosine_sim = util.pytorch_cos_sim(DP_002, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['DP02_01','DP02_02','DP02_03','DP02_04']
cosine_sim.to_csv(DATA_OUT_PATH + 'DP02.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)

        DP02_01   DP02_02   DP02_03   DP02_04
0      0.126267  0.278907  0.113763  0.273333
1      0.446139  0.576367  0.499839  0.582871
2      0.276671  0.543921  0.296898  0.493337
3      0.266494  0.469672  0.306727  0.515525
4      0.248290  0.442514  0.197079  0.349398
...         ...       ...       ...       ...
34809  0.158613  0.297703  0.208720  0.310121
34810  0.402814  0.505673  0.453960  0.469843
34811  0.363493  0.365104  0.424007  0.368434
34812  0.174101  0.328431  0.228908  0.205560
34813  0.240409  0.279661  0.274803  0.205213

[34814 rows x 4 columns]


In [25]:
cosine_sim = util.pytorch_cos_sim(DP_003, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['DP03_01','DP03_02','DP03_03','DP03_04']
cosine_sim.to_csv(DATA_OUT_PATH + 'DP03.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)

        DP03_01   DP03_02   DP03_03   DP03_04
0      0.169027  0.263265  0.098411  0.272824
1      0.536400  0.552031  0.451535  0.620985
2      0.389437  0.383299  0.139344  0.384182
3      0.347283  0.398166  0.202024  0.427785
4      0.342146  0.380963  0.138606  0.319352
...         ...       ...       ...       ...
34809  0.225905  0.421801  0.277456  0.367364
34810  0.376168  0.445365  0.327226  0.380472
34811  0.376104  0.296404  0.320004  0.398873
34812  0.287475  0.330806  0.190048  0.341250
34813  0.331697  0.359326  0.356266  0.412163

[34814 rows x 4 columns]


In [27]:
cosine_sim = util.pytorch_cos_sim(DP_004, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['DP04_01','DP04_02','DP04_03','DP04_04']
cosine_sim.to_csv(DATA_OUT_PATH + 'DP04.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)

        DP04_01   DP04_02   DP04_03   DP04_04
0      0.214621  0.367588  0.224974  0.201009
1      0.678569  0.584163  0.505265  0.533833
2      0.498829  0.442428  0.301718  0.350856
3      0.466770  0.479136  0.373212  0.371747
4      0.405879  0.372733  0.241638  0.252960
...         ...       ...       ...       ...
34809  0.340753  0.410831  0.352442  0.262140
34810  0.568742  0.504438  0.406045  0.361041
34811  0.389790  0.393271  0.410166  0.278445
34812  0.280549  0.283066  0.282462  0.223845
34813  0.307392  0.263231  0.279049  0.211221

[34814 rows x 4 columns]


In [29]:
cosine_sim = util.pytorch_cos_sim(OCI_01_001, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['OCI_H_1','OCI_H_2','OCI_H_3','OCI_H_4','OCI_H_5','OCI_H_6','OCI_H_7','OCI_H_8','OCI_H_9','OCI_H_10']
cosine_sim.to_csv(DATA_OUT_PATH + 'OCI_01_01.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)

        OCI_H_1   OCI_H_2   OCI_H_3   OCI_H_4   OCI_H_5   OCI_H_6   OCI_H_7  \
0      0.141818  0.186825  0.122748 -0.000182  0.225583 -0.030554  0.163258   
1      0.372389  0.316990  0.317930  0.092264  0.407687  0.203242  0.429171   
2      0.244424  0.200539  0.207586  0.120649  0.383918  0.106520  0.255809   
3      0.290266  0.223803  0.194459  0.041291  0.382925  0.142306  0.284440   
4      0.229012  0.339351  0.307518  0.042365  0.428526  0.150885  0.247887   
...         ...       ...       ...       ...       ...       ...       ...   
34809  0.422642  0.424566  0.284415  0.134472  0.325268  0.220038  0.404047   
34810  0.350750  0.202259  0.320275  0.030486  0.470569  0.221045  0.239688   
34811  0.408866  0.181162  0.202957  0.114870  0.377793  0.266129  0.282269   
34812  0.409613  0.261325  0.413597  0.140881  0.336843  0.141756  0.384952   
34813  0.448476  0.227834  0.297347  0.109435  0.155793  0.262022  0.392724   

        OCI_H_8   OCI_H_9  OCI_H_10  
0      0.1123

In [31]:
cosine_sim = util.pytorch_cos_sim(OCI_01_002, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['OCI_AF_1','OCI_AF_2','OCI_AF_3','OCI_AF_4','OCI_AF_5','OCI_AF_6','OCI_AF_7','OCI_AF_8','OCI_AF_9','OCI_AF_10']
cosine_sim.to_csv(DATA_OUT_PATH + 'OCI_01_02.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)

       OCI_AF_1  OCI_AF_2  OCI_AF_3  OCI_AF_4  OCI_AF_5  OCI_AF_6  OCI_AF_7  \
0      0.091598  0.192424  0.058616  0.187555  0.050453  0.201950  0.142335   
1      0.211743  0.268360  0.240619  0.568275  0.146557  0.282618  0.269512   
2      0.136865  0.302968  0.222306  0.414223  0.178138  0.297710  0.244557   
3      0.154087  0.273889  0.209264  0.506099  0.134494  0.371755  0.293116   
4      0.148427  0.287879  0.130622  0.290426  0.137922  0.167484  0.211629   
...         ...       ...       ...       ...       ...       ...       ...   
34809  0.226199  0.316835  0.368476  0.501436  0.263579  0.400920  0.364632   
34810  0.128597  0.261981  0.276453  0.503155  0.148436  0.286690  0.288568   
34811  0.171329  0.285571  0.367721  0.586044  0.135444  0.365332  0.350196   
34812  0.250323  0.324186  0.148939  0.265253  0.146112  0.304893  0.324080   
34813  0.339926  0.259256  0.201091  0.371915  0.253388  0.267154  0.333426   

       OCI_AF_8  OCI_AF_9  OCI_AF_10  
0      0.242

In [33]:
cosine_sim = util.pytorch_cos_sim(OCI_01_003, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['OCI_AC_1','OCI_AC_2','OCI_AC_3','OCI_AC_4','OCI_AC_5','OCI_AC_6','OCI_AC_7','OCI_AC_8','OCI_AC_9','OCI_AC_10']
cosine_sim.to_csv(DATA_OUT_PATH + 'OCI_01_03.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)

       OCI_AC_1  OCI_AC_2  OCI_AC_3  OCI_AC_4  OCI_AC_5  OCI_AC_6  OCI_AC_7  \
0      0.311875  0.195395  0.154282  0.131849  0.143538  0.194342  0.242154   
1      0.553575  0.450817  0.304150  0.397467  0.261760  0.108551  0.343251   
2      0.408102  0.245002  0.208915  0.243843  0.184232  0.066451  0.288403   
3      0.482719  0.312549  0.249967  0.352144  0.190815  0.060404  0.244841   
4      0.403133  0.252707  0.223649  0.202573  0.068226  0.068325  0.226700   
...         ...       ...       ...       ...       ...       ...       ...   
34809  0.459652  0.322975  0.274723  0.329370  0.331778  0.200933  0.173048   
34810  0.505495  0.395944  0.236937  0.412750  0.208964  0.087734  0.201852   
34811  0.407319  0.276315  0.200800  0.373388  0.231431  0.024215  0.172002   
34812  0.405445  0.358739  0.332017  0.170960  0.233758  0.239364  0.336368   
34813  0.167479  0.151323  0.271527  0.186196  0.138622  0.187975  0.204563   

       OCI_AC_8  OCI_AC_9  OCI_AC_10  
0      0.296

In [35]:
cosine_sim = util.pytorch_cos_sim(OCI_01_004, sentence_vectors_seicho)
cosine_sim= cosine_sim.numpy()
cosine_sim = cosine_sim.transpose()
cosine_sim= pd.DataFrame(cosine_sim)
cosine_sim.columns=['OCI_SA_1','OCI_SA_2','OCI_SA_3','OCI_SA_4','OCI_SA_5','OCI_SA_6','OCI_SA_7','OCI_SA_8','OCI_SA_9','OCI_SA_10']
cosine_sim.to_csv(DATA_OUT_PATH + 'OCI_01_04.csv', index=False, quoting=3, encoding="CP932") 
print(cosine_sim)

       OCI_SA_1  OCI_SA_2  OCI_SA_3  OCI_SA_4  OCI_SA_5  OCI_SA_6  OCI_SA_7  \
0      0.163238  0.316151  0.086415  0.055338  0.069220  0.280244  0.241588   
1      0.212275  0.435218  0.355298  0.262983  0.204337  0.258031  0.274239   
2      0.170494  0.447624  0.228985  0.097411  0.149162  0.337639  0.330389   
3      0.220744  0.521857  0.267195  0.157164  0.232884  0.246118  0.323347   
4      0.096223  0.372612  0.222501  0.088583  0.146608  0.270243  0.233968   
...         ...       ...       ...       ...       ...       ...       ...   
34809  0.229611  0.412893  0.272425  0.370571  0.244442  0.170643  0.158252   
34810  0.128236  0.361162  0.353293  0.166477  0.226193  0.241070  0.172609   
34811  0.199777  0.514213  0.396739  0.280916  0.300577  0.169557  0.276312   
34812  0.197459  0.345705  0.225151  0.105521  0.336665  0.376811  0.232041   
34813  0.198843  0.236177  0.302257  0.585148  0.273535  0.113282  0.133404   

       OCI_SA_8  OCI_SA_9  OCI_SA_10  
0      0.274